In [142]:
import pandas as pd
from fuzzywuzzy import fuzz
import jellyfish
import numpy as np
from geopy.geocoders import Nominatim, GoogleV3
import pickle
import spacy
import string
import operator

In [34]:
df = pd.read_csv("1hb_2014.csv")
print("List of columns for reference")
df.columns

List of columns for reference


Index(['lca_case_number', 'status', 'lca_case_submit', 'decision_date',
       'visa_class', 'lca_case_employment_start_date',
       'lca_case_employment_end_date', 'lca_case_employer_name',
       'lca_case_employer_address', 'lca_case_employer_city',
       'lca_case_employer_state', 'lca_case_employer_postal_code',
       'lca_case_soc_code', 'lca_case_soc_name', 'lca_case_job_title',
       'lca_case_wage_rate_from', 'lca_case_wage_rate_to',
       'lca_case_wage_rate_unit', 'full_time_pos', 'total_workers',
       'lca_case_workloc1_city', 'lca_case_workloc1_state', 'pw_1',
       'pw_unit_1', 'pw_source_1', 'other_wage_source_1', 'yr_source_pub_1',
       'lca_case_workloc2_city', 'lca_case_workloc2_state', 'pw_2',
       'pw_unit_2', 'pw_source_2', 'other_wage_source_2', 'yr_source_pub_2',
       'lca_case_naics_code'],
      dtype='object')

# Warm Up - Question 1

Problem definition:

1. Which companies applied for the largest number of H­1B visas where the job opening was located in NYC? Please describe any issues you may encounter summarizing the data by employer name.

Solution:

There were many issues with this dataset.  The first of these problems was just deciding on what is semantically considered New York City.  Technically New York City is all 5 borroughs, which should be fine.  But the fields `lca_case_workloc1_city` and `lca_case_workloc2_city` which stores the locations of the job openning have multiple values that map semantically to something that is reasonable to describe as New York City.

As you can see below, I enumerated all the possible choices for location in the 5 borroughs.  First I segmented the data into only New York state. 

In [40]:
first_work_location = df[df["lca_case_workloc1_state"] == "NY"]
second_work_location = df[df["lca_case_workloc2_state"] == "NY"]
loc1_cities = list(set(first_work_location["lca_case_workloc1_city"]))
loc2_cities = list(set(second_work_location["lca_case_workloc2_city"]))
total_cities = list(set(loc1_cities + loc2_cities))
total_cities = [city for city in total_cities if city is not np.nan]
len(total_cities)

765

Because the number of total city names was small it was relatively easy to figure out all locations that belonged in new york city proper. First I looked for any city names that had new in them and copied those names that clearly meant new york city into a list in a cell below and named the list `new_york_city_names`

In [41]:
for city in total_cities:
    if "NEW" in city:
        print(city)

NEW YROK
SLEEPY HOLLOW, NEW YORK
NEW YORK, 10003
NEWYORK
NEW WINDSOR
CITY OF NEW YORK
BRONX.NEW YORK
NEW PALTZ
NEW HYDE PARK
NEW YORK, NY -
NEW YORK NY
NEW HAMPTON
NEWBURGH
NEW BERLIN
METRO NEW YORK
NEW HARTFORD
NEW ORK
NEW YOK
MANHATTAN, NEW YORK
NEW YORK
NEW TORK
NEW YOUR
BRIDGEPORT / NEW YORK
NEW YORK CITY
NEWARK
BRONX,NEW YORK
NEWYORK CITY
NEW YORK, NEW YORK
NEW CITY
NEW YORK,NEW YORK
NEW
NEW YORK, NY
NEW WINDSOW
NEW YOURK
NEW ROCHELLE
NEW YORK,
NEW YORK CITY,
BRONX, NEW YORK


Then I looked around for other obvious New York City locations.  The next thing I did was get a bounding box for New York City from this site: https://www.mapdevelopers.com/geocode_bounding_box.php

Using that bounding box, I made use of geopy to map the city names, to the lat longs.  If the lat longs were inside the bounding box, then I considered adding it to the list:

In [62]:
google_api_key = pickle.load(open("google_geocoder_api.creds","rb"))
locations = []
google_encoder = GoogleV3(google_api_key)
nominatim_encoder = Nominatim()
city_indexes = []
for index, city in enumerate(total_cities):
    if "NY" not in city:
        text = city +",NY"
    else:
        text = city
    try:
        locations.append(google_encoder.geocode(text))
        city_indexes.append(index)
    except:
        locations.append(nominatim_encoder.geocode(text))
        city_indexes.append(index)

In [64]:
north_lat = 40.917577 
south_lat = 40.477399 
east_long = abs(-73.700272)
west_long = abs(-74.259090)
locations = [elem for elem in locations if elem]
for index,location in enumerate(locations):
    lat_check = False
    long_check = False
    if location.latitude < north_lat and location.latitude > south_lat:
        lat_check = True
    if abs(location.longitude) < west_long and abs(location.longitude) > east_long:
        long_check = True
    if lat_check and long_check:
        city_index = city_indexes[index]
        print(total_cities[city_index])

COLLEGE POINT
NEW YROK
DOUGLASTON
MASPETH
NEW YORK, 10003
SUITE 945
137 VARICK STREET, 2ND FLOOR
ELMHURST
NEWYORK
LONG ISLAND CITY
6TH FLOOR, ROOM 6027W
WHITETONE
GLEN OAKS
OAKLAND GARDENS
OZONE PARK
FLUSHING
JAMAICA
MIAMI
55 WATER STREET
CITY OF NEW YORK
LONG ISLANDY CITY
BRIARWOOD
LONGISLAND CITY
325 HUDSON STREET, 9TH FLOOR
BRONX,NY
BRONX.NEW YORK
CHELSEA
6TH FLOOR
160 CONVENT AVENUE
ROCHDALE VILLAGE
MT. VERNON
WOODSIDE
GREAT NECK,
NEW YORK, NY -
REGO
850 12TH AVENUE
LAURELTON
OAKLAND GARDEN
EAST ELMHURST
NEW YORK NY
BELLEVUE
43-34 32ND PLACE
LONG ISLAND CITY, QUEENS
GLENDALE
KEW GARDENS
WILLIAMSBURGH
METRO NEW YORK
RIVERDALE
BOWLING GREEN
SOUTH RICHMOND HILL
FRESH MEADOWS
ELMHRUST
WOODMERE
SUITE NO 1502
JAMAICA, NY
STATEN ISLAND
MIDDLE VILLAGE
LONG ISLAND CTY
JACKSON HEIGHTS
NEW ORK
MOUNT VERNON
1 COURT SQUARE
FOREST HILLS
NEW YOK
THROGGS NECK
CHICAGO
MANHATTAN, NEW YORK
SUITE 1802
ST. ALBANS
LIC
RIDGEWOOD
DOWNTOWN BROOKLYN
FOREST HILLS QUEENS
BOISE
JAMIACA
BRONX
SUITE 514
NEW YORK

From this list of 180 results - far smaller than the original lists length of 765, I googled or visually inspected each result.  If the result was a place I knew in New York City, I added it to the list.  If it didn't look familiar I googled it.  Those results that made it through this last test ended up in the list below, stored in a variable called `new_york_city_names`.

In [65]:
new_york_city_names = [
    "NEW YORK,", "BAYSIDE", "1 COURT SQUARE",
    "160 FORT WASHINGTON AVENUE", "ASTORIA",
    "LONG IS CITY", "RIDGEWOOD", "BROOKYLN",
    "160 CONVENT AVENUE", "227 EAST 56TH STREET",
    "SOUTH OZONE PARK", "BROOKLY", "LONG ISLAND CTY",
    "DOWNTOWN BROOKLYN", "BRONX,NEW YORK", "5 TIMES SQUARE",
    "MANHATTAN", "MANHATAN", "BROOKLYN,", "BROOKLN",
    "MEW YORK", "BRONX", "132 HARRISON PLACE",
    "LONGISLAND CITY", "NEW YORK", "MIDTOWN MANHATTAN",
    "METRO NEW YORK", "NEW YOUR", "BROOKYN",
    "NEW YORK, 10003", "BROOKLYN", "NEW TORK",
    "650 WEST 168TH STREET", "325 HUDSON STREET, 9TH FLOOR",
    "FORREST HILLS", "BRONX.NEW YORK", "CITY OF NEW YORK",
    "JACKSON HEIGHTS", "FLUSHING", "THROGGS NECK",
    "QUEENS VILLAGE", "NEW ORK", "JOHNSON STREET",
    "ROCKAWAY PARK", "LONG ISLAND CITY, QUEENS",
    "NEW YORK,NEW YORK", "MANHATTAN BEACH",
    "NEW YORK, NEW YORK", "NEW YORK CITY",
    "NEWYORK CITY", "S RICHMOND HILL",
    "L.I.C.", "RICHMOND HILL", "55 WATER STREET",
    "OZONE PARK", "LONG ISLANDY CITY", "JAMAICA",
    "INWOOD", "SUNNYSIDE", "NEW YORK NY",
    "CAMBRIA HEIGHTS", "NEW YORK CITY,",
    "THROGS NECK", "137 VARICK STREET, 2ND FLOOR",
    "#35620 OZONE PARK NY MSA", "JAMAICA, NY",
    "LONG ISLAND CITY,", "NEW YOURK", "NY",
    "NEWYORK", "YORKERS", "NEW YROK",
    "LIC", "13 W.100TH STREET, APT. 4B",
    "850 12TH AVENUE", "NEW YOK",
    "701 WEST 168TH STREET", "FLUSHING, NY",
    "630 WEST 168TH STREET", "FOREST HILLS",
    "BOWLING GREEN", "NEW YORK, NY -", "LITTLE NECK",
    "NEW YORK, NY", "43-34 32ND PLACE",
    "LONG ISLAD CITY", "REGO PARK",
    "MANHATTAN, NEW YORK", "BRONX,NY",
    "LONG ISLAND CITY", "NYC",
    "QUEENS", "YONKERS", "BRONX, NEW YORK"
]

len(new_york_city_names)

94

Next I segmented by city data by names I knew to be within new york city.  Anything not in the city was removed.  I visually checked to make sure the test worked by calling the first 5 entries.

In [121]:
def segment_to_nyc(x):
    if x in new_york_city_names:
        return True
    else:
        return False

nyc = df[df["lca_case_workloc1_city"].apply(segment_to_nyc)]
nyc.head()

,lca_case_number,status,lca_case_submit,decision_date,visa_class,lca_case_employment_start_date,lca_case_employment_end_date,lca_case_employer_name,lca_case_employer_address,lca_case_employer_city,...,other_wage_source_1,yr_source_pub_1,lca_case_workloc2_city,lca_case_workloc2_state,pw_2,pw_unit_2,pw_source_2,other_wage_source_2,yr_source_pub_2,lca_case_naics_code
23,I-200-09259-849209,CERTIFIED,2014-03-12,2014-03-18,H-1B,2014-09-10,2017-09-10,"FRS AMERICA, INC.",2700 LAKE COOK ROAD,RIVERWOODS,...,OFLC ONLINE DATA CENTER,2013.0,NEW YORK,NY,129418.0,Year,OES,OFLC ONLINE DATA CENTER,OFLC ONLINE DATA CENTER,511210.0
36,I-200-09300-229787,CERTIFIED-WITHDRAWN,2013-09-11,2014-04-10,H-1B,2014-02-28,2015-03-31,FITCH RATINGS,ONE STATE STREET PLAZA,NEW YORK,...,OFLC ONLINE DATA CENTER,2013.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,523930.0
43,I-200-09321-521235,CERTIFIED-WITHDRAWN,2012-05-02,2014-08-21,H-1B,2012-05-11,2015-05-10,BARCLAYS CAPITAL INC.,745 SEVENTH AVENUE,NEW YORK,...,OFLC ONLINE DATA CENTER,2011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,523110.0
62,I-200-10022-756568,CERTIFIED,2013-10-24,2013-11-01,H-1B,2013-10-24,2016-10-22,"BLACKROCK INVESTMENT MANAGEMENT, LLC",40 EAST 52ND STREET,NEW YORK,...,OFLC ONLINE DATA CENTER,2013.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,523920.0
98,I-200-10106-798993,CERTIFIED,2014-05-20,2014-05-27,H-1B,2014-10-01,2017-09-30,"F.O. USA, INC.",410 COLUMBUS AVENUE,NEW YORK,...,OFLC ONLINE DATA CENTER,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,448130.0


The next thing to do was look into employer names.  Knowing from the location data, that this data set is quiet messy I tried to come up with a scheme for canonicalizing the names of employers.  In order to do this efficiently I had to do a bit of messing around.  The first thing to note is the companies sometimes have different names, but refer to the same "actual" company.  I'm being a bit hand wavy here, but a good example of what I'm talking about presents itself with morgan stanley.  If you run the below code and then access the `employer_counts` dict like so:

```
>>> for elem in employer_counts:
...     if "MORGAN" in elem:
...             print(elem)
... 
MORGAN STANLEY SMITH BARNEY
MORGAN STANLEY BANK NA
MORGAN STANLEY CAPITAL GROUP
MORGAN STANLEY FUND SERVICES
MORGAN STANLEY INVESTMENT MANAGEMENT
MORGAN LEWIS BOCKIUS
MORGAN STANLEY BANK
MORGANE LE FAY
MORGANS HOTEL GROUP MANAGEMENT
```

We see that there are at least 4 businesses that are actually Morgan Stanley, as a lay person would think of it.  So it's probably best to be able to group by this natural categorization.

Digging in a bit to the code - one of the things you'll notice about `clean_split` - my preprocessing function, I remove some obvious words that are likely to appear quiet a bit.  Since these are company names, they add little value to the canonicalization process, therefore they are removed.  The next thing to note is my use of named entity recognition from `spacy`.  There entity tagger allows me to remove useless common words.  If the label "ORG" is attached to the entity, we simply include that in names to check.  

Stepping back a bit, to see what the overall goal of the code is - the high level goal is simple, determine which businesses occur multiple times, under slightly different names.  If we knew all the semantic organizations, like we know MORGAN STANLEY then we could simply check for that sub-string in all the other business names.  Unfortunately we don't, so we simply try to mimic that functionality generally, via these `ORG` entities we pulled out earlier.  Looping over each word, checking for each substring we believe to be an `ORG`, we are able to approximate what all the unique employer names are.  These are stored in the keys of `employer_counts`.  

In [144]:
def clean_split(employer):
    translations = [employer.maketrans(elem," ") for elem in string.punctuation]
    stop_words = ["INC", "LLC", "&", "PC", "LP", "LLP", "CO", "LTD", "CORP"]
    for translation in translations:
        employer = employer.translate(translation)
    employer = ''.join(employer)
    return [elem for elem in employer.split() if elem not in stop_words]
    
nlp = spacy.load('en')
nyc = df[df["lca_case_workloc1_city"].apply(segment_to_nyc)]
employers = list(nyc["lca_case_employer_name"].unique())
employer_counts = {}
for index, employer in enumerate(employers):
    other_employers = employers[:index] + employers[index+1:]
    names = clean_split(employer)
    names = [name for name in names if len(name) > 1]
    names = [name for name in names if not name.isdigit()]
    doc = nlp(' '.join(names))
    names = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    tmp = {}.fromkeys(names, 0)
    for name in names:
        tmp[name] += sum([employer.count(name) for employer in other_employers])
    employer_counts.update(tmp)

print(max(employer_counts.items(), key=operator.itemgetter(1)))

counter = 0
for employer in employer_counts:
    if employer_counts[employer] < 100:
        counter += 1
counter

('CA', 1357)


3256

Notice that some employer names occur quiet frequently - for instance, the most common is "CA".  Clearly "CA" is not a company name.  So we look for employers with counts less than 100 to try to thin out any obvious errors:

In [161]:
duplicates = []
uniques = []
for employer in employer_counts:
    if employer_counts[employer] == 1:
        uniques.append(employer)
    else:
        duplicates.append(employer)
print("Uniquely appearing companies",len(uniques))
print("Companies appearing more than once",len(duplicates))

Uniquely appearing companies 302
Companies appearing more than once 2958


From here we can see which companies appear more than once.  So all we need to do now is work over the duplicates, creating canonical names for each.  Then we can map the duplicate names to their canonical names.

In [185]:
def compare_companies(company_one, company_two):
    stop_words = ["INC", "LLC", "&", "PC", "LP", "LLP", "CO", "LTD", "CORP",
                "MANAGEMENT", "CAPITAL", "SERVICES", "BANK", "PLC", "FINANCIAL"
                 "NEW", "YORK", "DEPARTMENT", "UNIVERSITY"]
    company_one_words = [word for word in company_one.split() if word not in stop_words]
    company_two_words = [word for word in company_two.split() if word not in stop_words]
    common_words = 0
    for c1_word in company_one_words:
        for c2_word in company_two_words:
            if c1_word == c2_word:
                common_words += 1
    percent_common_words = common_words / len(company_two.split())
    if percent_common_words > 0.4:
        return True
    else:
        return False
    
mapping = {}
for index, company in enumerate(duplicates):
    if company not in list(mapping.keys()):
        other_companies = duplicates[:index] + duplicates[index+1:]
        for other_company in other_companies:
            if compare_companies(company, other_company):
                mapping[other_company] = company
len(mapping)

1081

In [184]:
keys = list(mapping.keys())
for ind, key in enumerate(keys):
    print(key, "==", mapping[key])
    if ind == 200:
        break

BARCLAYS SERVICES == BARCLAYS CAPITAL ENERGY
EURO RSCG NEW YORK == THE NEW YORK CITY DEPARTMENT OF HEALTH AND MENTAL
INTERNATIONAL HOUSE NEW YORK == THE NEW YORK CITY DEPARTMENT OF HEALTH AND MENTAL
THE BANK OF NEW YORK MELLON == THE NEW YORK CITY DEPARTMENT OF HEALTH AND MENTAL
DEUTSCHE BANK NEW YORK == THE NEW YORK CITY DEPARTMENT OF HEALTH AND MENTAL
NEW YORK UNIVERSITY HOSPITALS CENTER == NEW YORK UNIVERSITY SCHOOL OF MEDICINE
NEW == NEW METRO CONSTRUCTION
NEW YORK TECHNOLOGY PARTNERS == THE NEW YORK CITY DEPARTMENT OF HEALTH AND MENTAL
NEW YORK UNIVERSITY == THE NEW YORK CITY DEPARTMENT OF HEALTH AND MENTAL
YESHIVA UNIVERSITY == YESHIVA
TARDIS GROUP NEW YORK == THE NEW YORK CITY DEPARTMENT OF HEALTH AND MENTAL
THE CITY UNIVERSITY OF == THE NEW YORK CITY DEPARTMENT OF HEALTH AND MENTAL
NEW YORK FOUNDATION == THE NEW YORK CITY DEPARTMENT OF HEALTH AND MENTAL
FORDHAM UNIVERSITY == THE AMERICAN UNIVERSITY IN CAIRO
NEW YORK INSTITUTE OF ENGLISH AND BUSINESS == CHUNG TE INSTITUTE OF REL